# TASK 3 - PART 1: Saliency Mapping (The Smoking Gun)

## Objective

Identify **which specific words** in AI-generated text signal "AI" to our best detector (Tier B - 95% accuracy).

---

## Why Tier B (Not Tier C)?

**Decision Rationale:**

| Model | Test Accuracy | Reason to Use/Not Use |
|-------|---------------|----------------------|
| **Tier C (DistilBERT)** | 59% ❌ | Overfitted, predicts "Human" for 85.9% of errors |
| **Tier B (GloVe + NN)** | **95%** ✅ | **WINNER** - Reliable, captures semantic signals |
| **Tier A (Random Forest)** | 91% | Good, but focuses on structure not semantics |

**Key Point:** Task 3 asks for explainability of AI detection. Tier C (59%) explains overfitting, not AI detection. **Tier B (95%) explains what ACTUALLY works.**

**Scientific Justification:**
> "We analyze Tier B (95% accuracy) instead of Tier C (59% accuracy) because our goal is understanding successful AI detection, not debugging a failed model. Tier B's semantic approach provides actionable insights into vocabulary patterns that distinguish Victorian human text from modern AI text."

---

## The Approach

### What is Saliency Mapping?

**Saliency = Importance Score per Word**

For each word in a paragraph:
- **Positive score (red):** Pushes toward "AI" classification
- **Negative score (green):** Pushes toward "Human" classification
- **Zero score (neutral):** No contribution to decision

### Tool: SHAP (SHapley Additive exPlanations)

**Why SHAP?**
- ✅ Theoretically grounded (game theory, Shapley values)
- ✅ Works with any model (black-box approach)
- ✅ Word-level importance scores
- ✅ Beautiful visualizations (force plots, waterfall plots)
- ✅ Handles averaged embeddings well

**Alternative: Captum**
- Gradient-based (faster but requires gradients)
- PyTorch-specific
- Good for transformers, less ideal for averaged embeddings

### The Process

```
1. Load Tier B model (saved weights)
   ↓
2. Select "imposter" AI paragraphs (5-10 examples)
   ↓
3. For each paragraph:
   - Tokenize into words
   - Get GloVe embedding per word
   - Compute SHAP value per word
   - Map back to original text
   ↓
4. Visualize:
   - Color-coded HTML output
   - Force plots showing contribution flow
   - Top AI-signaling words
```

---

## Expected Results

**Hypothesis 1: Victorian vs Modern Vocabulary**
- Modern AI words ("tapestry," "delve," "navigate") → High AI scores
- Victorian words ("countenance," "parlour," "ere") → High Human scores

**Hypothesis 2: Temporal Semantic Gap**
- GloVe (trained on 2000s web) detects era-specific context
- 170-year Victorian→Modern gap is THE signal

**Hypothesis 3: AI-isms Are Real**
- Corporate jargon: "robust," "comprehensive," "landscape"
- AI clichés: "testament," "intricate," "nuanced"

---

Let's begin! 🚀

---

## Step 1: Install & Import Libraries

In [1]:
# Install SHAP (run once)
!pip install -q shap

In [2]:
# Core libraries
import numpy as np
import pandas as pd
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# SHAP for explainability
import shap

# PyTorch/Keras (for loading Tier B model)
import tensorflow as tf
from tensorflow import keras

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Random seed
np.random.seed(42)
tf.random.set_seed(42)

print("✅ All libraries imported successfully!")
print(f"   SHAP version: {shap.__version__}")
print(f"   TensorFlow version: {tf.__version__}")

2026-02-07 12:59:06.914728: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-07 12:59:06.949197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770449346.971116   27976 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770449346.977921   27976 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770449347.005524   27976 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

✅ All libraries imported successfully!
   SHAP version: 0.49.1
   TensorFlow version: 2.19.1


---

## Step 2: Load GloVe Embeddings

Same preprocessing as Tier B training.

In [3]:
# Load GloVe embeddings
GLOVE_PATH = Path('../glove/glove.6B.300d.txt')

print("Loading GloVe embeddings (this may take ~30 seconds)...")

glove_embeddings = {}
with open(GLOVE_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        glove_embeddings[word] = vector

EMBEDDING_DIM = 300

print(f"✅ GloVe embeddings loaded!")
print(f"   Vocabulary size: {len(glove_embeddings):,} words")
print(f"   Embedding dimension: {EMBEDDING_DIM}d")

Loading GloVe embeddings (this may take ~30 seconds)...
✅ GloVe embeddings loaded!
   Vocabulary size: 400,000 words
   Embedding dimension: 300d
✅ GloVe embeddings loaded!
   Vocabulary size: 400,000 words
   Embedding dimension: 300d


---

## Step 3: Define Text-to-Embedding Functions

Match Tier B preprocessing exactly.

In [4]:
def text_to_embedding(text, embeddings_dict):
    """
    Convert text to averaged GloVe embedding (300d vector).
    Same as Tier B training preprocessing.
    """
    # Tokenize (simple split + lowercase)
    words = text.lower().split()
    
    # Get embeddings for each word
    word_vectors = []
    for word in words:
        if word in embeddings_dict:
            word_vectors.append(embeddings_dict[word])
    
    # Average all word vectors
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        # If no words found, return zeros
        return np.zeros(EMBEDDING_DIM)


def get_word_embeddings(text, embeddings_dict):
    """
    Get individual word embeddings for SHAP analysis.
    Returns: List of (word, embedding) tuples.
    """
    words = text.lower().split()
    word_embeds = []
    
    for word in words:
        if word in embeddings_dict:
            word_embeds.append((word, embeddings_dict[word]))
        else:
            # Unknown word: use zero vector
            word_embeds.append((word, np.zeros(EMBEDDING_DIM)))
    
    return word_embeds


# Test functions
test_text = "The quick brown fox jumps over the lazy dog."
test_embed = text_to_embedding(test_text, glove_embeddings)
test_words = get_word_embeddings(test_text, glove_embeddings)

print(f"✅ Preprocessing functions defined!")
print(f"   Test text: '{test_text}'")
print(f"   Averaged embedding shape: {test_embed.shape}")
print(f"   Number of words: {len(test_words)}")

✅ Preprocessing functions defined!
   Test text: 'The quick brown fox jumps over the lazy dog.'
   Averaged embedding shape: (300,)
   Number of words: 9


---

## Step 4: Load Tier B Model

Load the saved Tier B model (95% accuracy).

In [ ]:
# Load Tier B model
MODEL_PATH = Path('../TASK2/task2_tier_b_glove_model.h5')

if MODEL_PATH.exists():
    model = keras.models.load_model(MODEL_PATH)
    print(f"✅ Tier B model loaded from: {MODEL_PATH}")
    print(f"\n📊 Model Architecture:")
    model.summary()
else:
    print(f"❌ Model file not found: {MODEL_PATH}")
    print(f"   Please ensure you've trained and saved Tier B model.")
    print(f"   Expected location: {MODEL_PATH.absolute()}")
    print(f"\n💡 To train the model, run all cells in:")
    print(f"   ../TASK2/task2_tier_b_semanticist.ipynb")

# Define label names (same as Tier B)
# NOTE: LabelEncoder sorts alphabetically, so:
#   Class 0 = AI_Styled (from class3.txt)
#   Class 1 = AI_Vanilla (from class2.txt)
#   Class 2 = Human (from class1_human.jsonl)
label_names = ['AI_Styled', 'AI_Vanilla', 'Human']

print(f"\n📋 Class Labels (alphabetically ordered by LabelEncoder):")
for i, name in enumerate(label_names):
    print(f"   {i}: {name}")


2026-02-06 11:18:43.308130: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✅ Tier B model loaded from: ../TASK2/task2_tier_b_glove_model.h5

📊 Model Architecture:


Model: "GloVe_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden1 (Dense)                 │ (None, 128)            │        38,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,981 (183.52 KB)

 Trainable params: 46,979 (183.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)


📋 Class Labels (alphabetically ordered by LabelEncoder):
   0: AI_Styled
   1: AI_Vanilla
   2: Human


---

## Step 5: Load Test Data

Get AI paragraphs that Tier B correctly classified.

In [7]:
# Load dataset
DATA_DIR = Path('../data/dataset')
CLASS2_FILE = DATA_DIR / 'class2.txt'  # AI Vanilla
CLASS3_FILE = DATA_DIR / 'class3.txt'  # AI Styled

# Load AI Vanilla
ai_vanilla_texts = []
if CLASS2_FILE.exists():
    with open(CLASS2_FILE, 'r', encoding='utf-8') as f:
        ai_vanilla_texts = [line.strip() for line in f if line.strip()]
    print(f"✅ AI Vanilla: {len(ai_vanilla_texts)} paragraphs loaded")
else:
    print(f"❌ AI Vanilla file not found: {CLASS2_FILE}")

# Load AI Styled
ai_styled_texts = []
if CLASS3_FILE.exists():
    with open(CLASS3_FILE, 'r', encoding='utf-8') as f:
        ai_styled_texts = [line.strip() for line in f if line.strip()]
    print(f"✅ AI Styled: {len(ai_styled_texts)} paragraphs loaded")
else:
    print(f"❌ AI Styled file not found: {CLASS3_FILE}")

print(f"\n📊 Total AI paragraphs: {len(ai_vanilla_texts) + len(ai_styled_texts)}")

✅ AI Vanilla: 300 paragraphs loaded
✅ AI Styled: 200 paragraphs loaded

📊 Total AI paragraphs: 500


---

## Step 6: Select "Imposter" Paragraphs for Analysis

Choose AI paragraphs that Tier B correctly identified.

In [8]:
# Select diverse examples
np.random.seed(42)

# Pick 5 AI Vanilla + 5 AI Styled
n_samples = 5

selected_ai_vanilla = np.random.choice(ai_vanilla_texts, size=min(n_samples, len(ai_vanilla_texts)), replace=False)
selected_ai_styled = np.random.choice(ai_styled_texts, size=min(n_samples, len(ai_styled_texts)), replace=False)

# Combine
imposter_paragraphs = list(selected_ai_vanilla) + list(selected_ai_styled)
imposter_labels = ['AI_Vanilla'] * len(selected_ai_vanilla) + ['AI_Styled'] * len(selected_ai_styled)

print(f"✅ Selected {len(imposter_paragraphs)} imposter paragraphs for analysis:")
print(f"   - AI Vanilla: {len(selected_ai_vanilla)}")
print(f"   - AI Styled: {len(selected_ai_styled)}")

# Preview first paragraph
print(f"\n📄 Example Imposter Paragraph (AI Vanilla):")
print(f"   {selected_ai_vanilla[0][:200]}...")

✅ Selected 10 imposter paragraphs for analysis:
   - AI Vanilla: 5
   - AI Styled: 5

📄 Example Imposter Paragraph (AI Vanilla):
   Ambition often masks a profound lack of self-acceptance. The drive to conquer the world frequently stems from a need to prove one’s worth to a critical parent or a doubting society. The high achiever ...


---

## Step 7: Verify Model Predictions

Ensure Tier B correctly identifies these as AI.

In [9]:
# Get embeddings for all imposter paragraphs
imposter_embeddings = np.array([text_to_embedding(text, glove_embeddings) for text in imposter_paragraphs])

# Get predictions
predictions = model.predict(imposter_embeddings)
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = [label_names[i] for i in predicted_classes]

# Get confidence scores
confidence_scores = np.max(predictions, axis=1)

# Create results dataframe
results_df = pd.DataFrame({
    'True_Label': imposter_labels,
    'Predicted_Label': predicted_labels,
    'Confidence': confidence_scores,
    'Correct': [true == pred for true, pred in zip(imposter_labels, predicted_labels)],
    'Text_Preview': [text[:80] + '...' for text in imposter_paragraphs]
})

print("📊 Model Predictions on Selected Imposters:\n")
print(results_df.to_string(index=False))

# Calculate accuracy
accuracy = results_df['Correct'].mean()
print(f"\n✅ Tier B Accuracy on Selected Imposters: {accuracy*100:.1f}%")
print(f"   (Expected: ~95% based on test set performance)")

# Filter to correctly classified for saliency analysis
correct_mask = results_df['Correct'].values
correctly_classified_texts = [text for text, correct in zip(imposter_paragraphs, correct_mask) if correct]
correctly_classified_labels = [label for label, correct in zip(imposter_labels, correct_mask) if correct]

print(f"\n🎯 Using {len(correctly_classified_texts)} correctly classified paragraphs for saliency analysis.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
📊 Model Predictions on Selected Imposters:

True_Label Predicted_Label  Confidence  Correct                                                                        Text_Preview
AI_Vanilla      AI_Vanilla    0.998500     True Ambition often masks a profound lack of self-acceptance. The drive to conquer th...
AI_Vanilla      AI_Vanilla    0.995679     True The concept of "home" is often a myth we spend our lives trying to reconstruct. ...
AI_Vanilla      AI_Vanilla    0.999585     True The myth of the "self-made" individual erases the complex network of support tha...
AI_Vanilla      AI_Vanilla    0.999887     True Ultimately, the endurance of love in a cynical world is a profound moral victory...
AI_Vanilla      AI_Vanilla    0.999831     True Redemption requires more than the passage of time. It demands active restitution...
 AI_Styled       AI_Styled    0.998999     True My treatment of Biddy stands as the monu

---

## Step 8: Implement Word-Level SHAP Explainer

**Challenge:** Tier B uses averaged embeddings (loses word-level info).

**Solution:** Create a wrapper that:
1. Takes individual word embeddings
2. Averages them
3. Feeds to model
4. SHAP can then attribute importance back to words

In [10]:
class WordLevelExplainer:
    """
    Wrapper to enable word-level explanations for averaged embedding model.
    """
    def __init__(self, model, embedding_dict):
        self.model = model
        self.embedding_dict = embedding_dict
        self.embedding_dim = 300
    
    def predict_from_words(self, word_list):
        """
        Predict from list of words.
        Args:
            word_list: List of words (already tokenized)
        Returns:
            Predictions (batch_size, 3)
        """
        # Get embeddings for each word
        embeddings = []
        for word in word_list:
            if word in self.embedding_dict:
                embeddings.append(self.embedding_dict[word])
            else:
                embeddings.append(np.zeros(self.embedding_dim))
        
        # Average embeddings
        if len(embeddings) > 0:
            avg_embedding = np.mean(embeddings, axis=0)
        else:
            avg_embedding = np.zeros(self.embedding_dim)
        
        # Reshape for model (add batch dimension)
        avg_embedding = avg_embedding.reshape(1, -1)
        
        # Get prediction
        return self.model.predict(avg_embedding, verbose=0)
    
    def predict_batch(self, word_lists):
        """
        Predict for batch of word lists.
        Args:
            word_lists: List of word lists
        Returns:
            Predictions array (batch_size, 3)
        """
        predictions = []
        for word_list in word_lists:
            pred = self.predict_from_words(word_list)
            predictions.append(pred[0])  # Remove batch dimension
        return np.array(predictions)


# Create explainer
word_explainer = WordLevelExplainer(model, glove_embeddings)

# Test it
test_words = test_text.lower().split()
test_pred = word_explainer.predict_from_words(test_words)

print(f"✅ Word-level explainer created!")
print(f"   Test prediction shape: {test_pred.shape}")
print(f"   Predicted class: {label_names[np.argmax(test_pred)]}")
print(f"   Confidence scores: {test_pred[0]}")

✅ Word-level explainer created!
   Test prediction shape: (1, 3)
   Predicted class: Human
   Confidence scores: [3.2798994e-02 1.8192646e-04 9.6701908e-01]


---

## Step 9: Generate SHAP Values (Word-Level Importance)

**This may take 2-5 minutes** depending on paragraph length.

In [16]:
# Select first correctly classified paragraph for detailed analysis
if len(correctly_classified_texts) > 0:
    sample_text = correctly_classified_texts[0]
    sample_label = correctly_classified_labels[0]
    
    print(f"🔍 Analyzing Sample Paragraph ({sample_label}):\n")
    print(f"{sample_text}\n")
    print("=" * 80)
    
    # Tokenize
    words = sample_text.lower().split()
    print(f"\n📊 Paragraph Statistics:")
    print(f"   Total words: {len(words)}")
    print(f"   Unique words: {len(set(words))}")
    
    # Get model prediction
    pred = word_explainer.predict_from_words(words)
    predicted_class = np.argmax(pred)
    confidence = pred[0][predicted_class]
    
    print(f"\n🎯 Model Prediction:")
    print(f"   Predicted: {label_names[predicted_class]}")
    print(f"   Confidence: {confidence*100:.2f}%")
    print(f"   Class probabilities:")
    for i, name in enumerate(label_names):
        print(f"      {name}: {pred[0][i]*100:.2f}%")
    
    print(f"\n⏳ Computing SHAP values (this may take 1-2 minutes)...")
    
    # Create SHAP explainer
    # Use KernelExplainer (model-agnostic, works with any black-box model)
    # Background data: sample of common word combinations (as word lists)
    background_word_lists = [
        "the quick brown fox".split(),
        "a very simple sentence".split(),
        "human writing is natural".split(),
        "artificial intelligence generated text".split(),
        "victorian literature style prose".split()
    ]
    
    # Convert background data to embeddings for SHAP
    # SHAP needs numerical data, not raw words
    background_embeddings = np.array([
        text_to_embedding(' '.join(word_list), glove_embeddings) 
        for word_list in background_word_lists
    ])
    
    print(f"   Background data shape: {background_embeddings.shape}")
    
    # Create a model wrapper that takes embeddings directly
    def model_predict_from_embeddings(embeddings):
        """
        Predict from embeddings array (for SHAP).
        Args:
            embeddings: numpy array of shape (batch_size, 300)
        Returns:
            predictions array (batch_size, 3)
        """
        return model.predict(embeddings, verbose=0)
    
    # Create SHAP explainer with embeddings
    explainer = shap.KernelExplainer(
        model_predict_from_embeddings,
        background_embeddings,
        link="identity"
    )
    
    # Convert our sample paragraph to embedding
    sample_embedding = text_to_embedding(sample_text, glove_embeddings).reshape(1, -1)
    
    print(f"   Sample embedding shape: {sample_embedding.shape}")
    
    # Compute SHAP values for the paragraph-level embedding
    shap_values = explainer.shap_values(sample_embedding)
    
    print(f"✅ SHAP values computed!")
    
    # Debug: Check the actual shape
    shap_array = np.array(shap_values)
    print(f"   SHAP shape: {shap_array.shape}")
    
    # SHAP can return different formats:
    # - List of arrays: [class0_shap, class1_shap, class2_shap] each (n_samples, n_features)
    # - Single array: (n_samples, n_features, n_classes)
    
    # Normalize to consistent format: (n_samples, n_features, n_classes)
    if isinstance(shap_values, list):
        # Format: list of arrays, stack them
        shap_array = np.stack(shap_values, axis=-1)  # Shape: (n_samples, n_features, n_classes)
        print(f"   Converted list to array shape: {shap_array.shape}")
    else:
        # Already an array
        shap_array = np.array(shap_values)
        # Check if shape is (n_samples, n_features, n_classes) or (n_features, n_classes)
        if len(shap_array.shape) == 2:
            # Shape is (n_features, n_classes), add sample dimension
            shap_array = shap_array[np.newaxis, ...]
            print(f"   Added sample dimension: {shap_array.shape}")
    
    # Now shap_array should be (1, 300, 3) or (n_samples, 300, 3)
    # Extract first sample: (300, 3)
    shap_per_feature = shap_array[0]  # Shape: (300, 3)
    print(f"   SHAP per embedding dimension: {shap_per_feature.shape}")
    
    # Now we need to map embedding-level importance back to words
    # Method: For each word, compute its contribution to the final embedding
    print(f"\n🔍 Mapping SHAP values back to words...")
    
    # Get individual word embeddings
    word_embeddings_list = []
    for word in words:
        if word in glove_embeddings:
            word_embeddings_list.append(glove_embeddings[word])
        else:
            word_embeddings_list.append(np.zeros(EMBEDDING_DIM))
    
    word_embeddings_array = np.array(word_embeddings_list)  # Shape: (num_words, 300)
    
    # For each word, compute its importance for each class
    # Method: dot product of word embedding with SHAP values for each class
    # Then divide by number of words (since we averaged)
    word_importance_scores = []
    
    for word_emb in word_embeddings_array:
        # Compute importance for each class
        class_importances = []
        for class_idx in range(3):
            # Get SHAP values for this class across all 300 dimensions
            class_shap = shap_per_feature[:, class_idx]  # Shape: (300,)
            
            # Word contribution: dot product of word embedding and SHAP values
            # Then divide by number of words (since we averaged)
            contribution = np.dot(word_emb, class_shap) / len(words)
            
            class_importances.append(contribution)
        word_importance_scores.append(class_importances)
    
    word_importance_scores = np.array(word_importance_scores)  # Shape: (num_words, 3)
    
    print(f"✅ Word-level importance computed!")
    print(f"   Shape: {word_importance_scores.shape}")
    print(f"   (words, classes)")
    
else:
    print("❌ No correctly classified paragraphs found for analysis.")
    print("   Check if model loaded correctly and test data is available.")

🔍 Analyzing Sample Paragraph (AI_Vanilla):

Ambition often masks a profound lack of self-acceptance. The drive to conquer the world frequently stems from a need to prove one’s worth to a critical parent or a doubting society. The high achiever confuses being admired with being loved. They collect accolades like armor, believing that if they build a fortress of success, they will finally feel safe. However, external achievements rarely fill internal voids. The trophy on the shelf cannot silence the critic in the head. True peace comes only when the individual realizes that their value exists independently of their resume.


📊 Paragraph Statistics:
   Total words: 94
   Unique words: 74

🎯 Model Prediction:
   Predicted: AI_Vanilla
   Confidence: 99.70%
   Class probabilities:
      AI_Styled: 0.30%
      AI_Vanilla: 99.70%
      Human: 0.00%

⏳ Computing SHAP values (this may take 1-2 minutes)...
   Background data shape: (5, 300)
   Sample embedding shape: (1, 300)


  0%|          | 0/1 [00:00<?, ?it/s]

✅ SHAP values computed!
   SHAP shape: (1, 300, 3)
   SHAP per embedding dimension: (300, 3)

🔍 Mapping SHAP values back to words...
✅ Word-level importance computed!
   Shape: (94, 3)
   (words, classes)


---

## Step 10: Visualize Word-Level Importance

Create color-coded visualization showing which words signal "AI".

In [17]:
# Extract word importance for predicted class
# word_importance_scores shape: (num_words, 3) - one score per class
# We want the scores for the PREDICTED class

target_class = label_names[predicted_class]

# Get importance scores for the predicted class
word_importance = word_importance_scores[:, predicted_class]

# Create dataframe
word_importance_df = pd.DataFrame({
    'Word': words,
    'SHAP_Value': word_importance,
    'Abs_SHAP': np.abs(word_importance)
})

# Sort by absolute importance
word_importance_df = word_importance_df.sort_values('Abs_SHAP', ascending=False)

print(f"\n📊 TOP 20 MOST IMPORTANT WORDS (for predicting '{target_class}'):\n")
print("=" * 80)
print(f"{'Word':<20} {'SHAP Value':<15} {'Effect'}")
print("=" * 80)

for idx, row in word_importance_df.head(20).iterrows():
    word = row['Word']
    shap_val = row['SHAP_Value']
    
    if shap_val > 0:
        effect = f"→ AI (strength: {shap_val:.4f})"
        marker = "🔴"
    else:
        effect = f"→ Human (strength: {abs(shap_val):.4f})"
        marker = "🟢"
    
    print(f"{marker} {word:<18} {shap_val:>+.6f}     {effect}")

print("\n" + "=" * 80)
print(f"\n💡 Interpretation:")
print(f"   🔴 Positive SHAP = Word pushes toward AI classification")
print(f"   🟢 Negative SHAP = Word pushes toward Human classification")
print(f"   Larger magnitude = Stronger influence")


📊 TOP 20 MOST IMPORTANT WORDS (for predicting 'AI_Vanilla'):

Word                 SHAP Value      Effect
🔴 believing          +0.001500     → AI (strength: 0.0015)
🔴 fortress           +0.001384     → AI (strength: 0.0014)
🔴 value              +0.001356     → AI (strength: 0.0014)
🔴 build              +0.001094     → AI (strength: 0.0011)
🔴 true               +0.001077     → AI (strength: 0.0011)
🔴 prove              +0.000831     → AI (strength: 0.0008)
🔴 individual         +0.000739     → AI (strength: 0.0007)
🔴 doubting           +0.000722     → AI (strength: 0.0007)
🔴 conquer            +0.000721     → AI (strength: 0.0007)
🟢 critic             -0.000699     → Human (strength: 0.0007)
🟢 high               -0.000668     → Human (strength: 0.0007)
🟢 a                  -0.000658     → Human (strength: 0.0007)
🟢 a                  -0.000658     → Human (strength: 0.0007)
🟢 a                  -0.000658     → Human (strength: 0.0007)
🟢 a                  -0.000658     → Human (strength

---

## Step 11: Create HTML Color-Coded Visualization

Generate beautiful HTML output with color-coded words.

In [18]:
from IPython.display import HTML

def create_colored_text(words, shap_values, max_color_intensity=0.5):
    """
    Create HTML with color-coded words based on SHAP values.
    Red = AI signal, Green = Human signal
    """
    # Normalize SHAP values for color intensity
    max_abs_shap = np.max(np.abs(shap_values))
    if max_abs_shap == 0:
        max_abs_shap = 1  # Avoid division by zero
    
    html = '<div style="line-height: 2.0; font-size: 16px; font-family: Arial, sans-serif;">'
    
    for word, shap_val in zip(words, shap_values):
        # Calculate color intensity (0 to max_color_intensity)
        intensity = min(abs(shap_val) / max_abs_shap * max_color_intensity, max_color_intensity)
        
        if shap_val > 0:  # Pushes toward AI
            # Red background
            bg_color = f'rgba(255, 0, 0, {intensity})'
            title = f'AI signal: +{shap_val:.4f}'
        elif shap_val < 0:  # Pushes toward Human
            # Green background
            bg_color = f'rgba(0, 255, 0, {intensity})'
            title = f'Human signal: {shap_val:.4f}'
        else:  # Neutral
            bg_color = 'transparent'
            title = 'Neutral'
        
        html += f'<span style="background-color: {bg_color}; padding: 2px 4px; margin: 2px; border-radius: 3px;" title="{title}">{word}</span> '
    
    html += '</div>'
    return html


# Create visualization
html_output = create_colored_text(words, word_importance)

print("\n📊 COLOR-CODED SALIENCY MAP:\n")
print("=" * 80)
print("\n🔴 Red = AI signal | 🟢 Green = Human signal | ⚪ Neutral\n")
print("Hover over words to see exact SHAP values!\n")
print("=" * 80)

# Display HTML
display(HTML(html_output))

# Save to file
output_file = Path('../results/visualizations/saliency_map_sample.html')
output_file.parent.mkdir(parents=True, exist_ok=True)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>TASK 3 - Saliency Map</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                max-width: 1000px;
                margin: 50px auto;
                padding: 20px;
            }}
            h1 {{
                color: #333;
            }}
            .legend {{
                margin: 20px 0;
                padding: 15px;
                background-color: #f5f5f5;
                border-radius: 5px;
            }}
            .stats {{
                margin: 20px 0;
                padding: 15px;
                background-color: #e8f4f8;
                border-radius: 5px;
            }}
        </style>
    </head>
    <body>
        <h1>🔍 TASK 3: Saliency Mapping - Word-Level AI Detection</h1>
        
        <div class="stats">
            <h3>📊 Prediction Statistics</h3>
            <p><strong>True Label:</strong> {sample_label}</p>
            <p><strong>Predicted:</strong> {target_class}</p>
            <p><strong>Confidence:</strong> {confidence*100:.2f}%</p>
        </div>
        
        <div class="legend">
            <h3>Legend</h3>
            <p>🔴 <span style="background-color: rgba(255, 0, 0, 0.3); padding: 2px 8px; border-radius: 3px;">Red</span> = Word signals AI authorship</p>
            <p>🟢 <span style="background-color: rgba(0, 255, 0, 0.3); padding: 2px 8px; border-radius: 3px;">Green</span> = Word signals Human authorship</p>
            <p>⚪ No color = Neutral word (minimal contribution)</p>
            <p><em>Hover over words to see exact SHAP values!</em></p>
        </div>
        
        <h3>Analyzed Paragraph:</h3>
        {html_output}
        
        <hr style="margin: 40px 0;">
        <p style="color: #666; font-size: 14px;">Generated by TASK 3 - Explainability Analysis | Tier B Model (95% accuracy)</p>
    </body>
    </html>
    """)

print(f"\n✅ Saliency map saved to: {output_file}")
print(f"   Open this file in a browser to see interactive visualization!")


📊 COLOR-CODED SALIENCY MAP:


🔴 Red = AI signal | 🟢 Green = Human signal | ⚪ Neutral

Hover over words to see exact SHAP values!




✅ Saliency map saved to: ../results/visualizations/saliency_map_sample.html
   Open this file in a browser to see interactive visualization!


---

## Step 12: Identify Top AI-Signaling Words Across All Samples

Analyze all correctly classified paragraphs to find common AI-isms.

In [20]:
print("🔍 Analyzing ALL correctly classified paragraphs...")
print(f"   Total paragraphs to analyze: {len(correctly_classified_texts)}\n")
print("   ⏳ This may take 5-10 minutes...\n")

# Store all word importance scores
all_word_scores = {}

for idx, (text, true_label) in enumerate(zip(correctly_classified_texts[:5], correctly_classified_labels[:5])):
    print(f"   Processing paragraph {idx+1}/{min(5, len(correctly_classified_texts))}...", end="")
    
    # Tokenize
    words_list = text.lower().split()
    
    # Get prediction
    pred = word_explainer.predict_from_words(words_list)
    pred_class = np.argmax(pred)
    
    # Convert paragraph to embedding
    para_embedding = text_to_embedding(text, glove_embeddings).reshape(1, -1)
    
    # Compute SHAP values for this paragraph
    shap_vals = explainer.shap_values(para_embedding)
    
    # Normalize SHAP format (same as Step 9)
    shap_array = np.array(shap_vals)
    if isinstance(shap_vals, list):
        shap_array = np.stack(shap_vals, axis=-1)
    else:
        if len(shap_array.shape) == 2:
            shap_array = shap_array[np.newaxis, ...]
    
    # Extract first sample: (300, 3)
    shap_per_feature = shap_array[0]
    
    # Map SHAP values back to words
    word_embs = []
    for word in words_list:
        if word in glove_embeddings:
            word_embs.append(glove_embeddings[word])
        else:
            word_embs.append(np.zeros(EMBEDDING_DIM))
    word_embs = np.array(word_embs)
    
    # Compute word importance for predicted class
    # Get SHAP values for predicted class across all 300 dimensions
    class_shap = shap_per_feature[:, pred_class]  # Shape: (300,)
    word_scores = np.dot(word_embs, class_shap) / len(words_list)
    
    # Store word scores
    for word, score in zip(words_list, word_scores):
        if word not in all_word_scores:
            all_word_scores[word] = []
        all_word_scores[word].append(score)
    
    print(" ✅")

print("\n✅ Analysis complete!\n")

# Calculate average scores per word
word_avg_scores = {
    word: {
        'mean': np.mean(scores),
        'count': len(scores),
        'std': np.std(scores)
    }
    for word, scores in all_word_scores.items()
}

# Convert to dataframe
word_stats_df = pd.DataFrame([
    {'Word': word, **stats}
    for word, stats in word_avg_scores.items()
])

# Filter: Words that appear at least 2 times
word_stats_df = word_stats_df[word_stats_df['count'] >= 2]

# Sort by mean SHAP value (positive = AI signal)
word_stats_df = word_stats_df.sort_values('mean', ascending=False)

print("="*80)
print("TOP 30 AI-SIGNALING WORDS (across all analyzed paragraphs)")
print("="*80)
print(f"{'Rank':<6} {'Word':<20} {'Avg SHAP':<12} {'Count':<8} {'Effect'}")
print("="*80)

for rank, (idx, row) in enumerate(word_stats_df.head(30).iterrows(), 1):
    word = row['Word']
    mean_shap = row['mean']
    count = int(row['count'])
    
    if mean_shap > 0:
        effect = "Strong AI signal 🔴"
    else:
        effect = "Human signal 🟢"
    
    print(f"{rank:<6} {word:<20} {mean_shap:>+.6f}    {count:<8} {effect}")

print("\n" + "="*80)

# Save results
output_csv = Path('../results/top_ai_words.csv')
word_stats_df.to_csv(output_csv, index=False)
print(f"\n✅ Full results saved to: {output_csv}")

🔍 Analyzing ALL correctly classified paragraphs...
   Total paragraphs to analyze: 10

   ⏳ This may take 5-10 minutes...

   Processing paragraph 1/5...

  0%|          | 0/1 [00:00<?, ?it/s]

 ✅
   Processing paragraph 2/5...

  0%|          | 0/1 [00:00<?, ?it/s]

 ✅
   Processing paragraph 3/5...

...

 ✅
   Processing paragraph 4/5...

  0%|          | 0/1 [00:00<?, ?it/s]

 ✅
   Processing paragraph 5/5...

  0%|          | 0/1 [00:00<?, ?it/s]

 ✅

✅ Analysis complete!

TOP 30 AI-SIGNALING WORDS (across all analyzed paragraphs)
Rank   Word                 Avg SHAP     Count    Effect
1      build                +0.001251    2        Strong AI signal 🔴
2      value                +0.001086    2        Strong AI signal 🔴
3      place                +0.000902    2        Strong AI signal 🔴
4      redemption           +0.000826    3        Strong AI signal 🔴
5      myth                 +0.000811    2        Strong AI signal 🔴
6      as                   +0.000758    4        Strong AI signal 🔴
7      their                +0.000558    6        Strong AI signal 🔴
8      profound             +0.000547    2        Strong AI signal 🔴
9      human                +0.000545    3        Strong AI signal 🔴
10     our                  +0.000465    3        Strong AI signal 🔴
11     an                   +0.000352    2        Strong AI signal 🔴
12     labor                +0.000298    2        Strong AI signal 🔴
13     in                   +0

---

## Step 13: Categorize Words by Type

Are they AI-isms, Victorian words, or structural markers?

In [21]:
# Define word categories
AI_CLICHES = [
    'tapestry', 'delve', 'testament', 'navigate', 'landscape', 
    'robust', 'comprehensive', 'intricate', 'nuanced', 'leverage',
    'framework', 'paradigm', 'multifaceted', 'holistic', 'resonate',
    'transformative', 'dynamic', 'innovative', 'strategic', 'synergy'
]

VICTORIAN_WORDS = [
    'countenance', 'parlour', 'hitherto', 'ere', 'whence', 'thee',
    'thou', 'thy', 'therein', 'thereof', 'wherein', 'henceforward',
    'forthwith', 'betwixt', 'heretofore', 'perchance'
]

# Categorize top AI-signaling words
top_ai_words = word_stats_df[word_stats_df['mean'] > 0].head(50)

ai_cliches_found = []
victorian_found = []
other_words = []

for idx, row in top_ai_words.iterrows():
    word = row['Word']
    if word in AI_CLICHES:
        ai_cliches_found.append((word, row['mean'], row['count']))
    elif word in VICTORIAN_WORDS:
        victorian_found.append((word, row['mean'], row['count']))
    else:
        other_words.append((word, row['mean'], row['count']))

print("="*80)
print("WORD CATEGORIZATION ANALYSIS")
print("="*80)

print(f"\n🤖 AI CLICHÉS DETECTED ({len(ai_cliches_found)} found):\n")
if ai_cliches_found:
    for word, shap, count in ai_cliches_found:
        print(f"   🔴 '{word}' - SHAP: {shap:+.6f} (appears {int(count)}x)")
else:
    print("   None found in top 50 AI-signaling words.")

print(f"\n📚 VICTORIAN VOCABULARY ({len(victorian_found)} found):\n")
if victorian_found:
    for word, shap, count in victorian_found:
        print(f"   🟢 '{word}' - SHAP: {shap:+.6f} (appears {int(count)}x)")
else:
    print("   None found in top 50 AI-signaling words.")

print(f"\n📝 OTHER HIGH-IMPORTANCE WORDS (Top 10):\n")
for word, shap, count in other_words[:10]:
    print(f"   • '{word}' - SHAP: {shap:+.6f} (appears {int(count)}x)")

print("\n" + "="*80)
print("\n💡 KEY FINDING:")
if len(ai_cliches_found) > 0:
    print(f"   ✅ AI-isms ARE detectable! Found {len(ai_cliches_found)} known AI clichés.")
else:
    print(f"   ⚠️ Known AI-isms not prominent. Model relies on OTHER semantic signals.")
    print(f"   → Likely detecting: Era-specific context, not specific buzzwords.")
print("="*80)

WORD CATEGORIZATION ANALYSIS

🤖 AI CLICHÉS DETECTED (0 found):

   None found in top 50 AI-signaling words.

📚 VICTORIAN VOCABULARY (0 found):

   None found in top 50 AI-signaling words.

📝 OTHER HIGH-IMPORTANCE WORDS (Top 10):

   • 'build' - SHAP: +0.001251 (appears 2x)
   • 'value' - SHAP: +0.001086 (appears 2x)
   • 'place' - SHAP: +0.000902 (appears 2x)
   • 'redemption' - SHAP: +0.000826 (appears 3x)
   • 'myth' - SHAP: +0.000811 (appears 2x)
   • 'as' - SHAP: +0.000758 (appears 4x)
   • 'their' - SHAP: +0.000558 (appears 6x)
   • 'profound' - SHAP: +0.000547 (appears 2x)
   • 'human' - SHAP: +0.000545 (appears 3x)
   • 'our' - SHAP: +0.000465 (appears 3x)


💡 KEY FINDING:
   ⚠️ Known AI-isms not prominent. Model relies on OTHER semantic signals.
   → Likely detecting: Era-specific context, not specific buzzwords.


---

## Summary & Next Steps

### What We've Done ✅

1. **Loaded Tier B model** (95% accuracy - the winner)
2. **Selected imposter paragraphs** (AI text correctly classified)
3. **Implemented word-level SHAP explainer** (overcame averaged embedding challenge)
4. **Generated saliency maps** (color-coded word importance)
5. **Identified top AI-signaling words** (across multiple paragraphs)
6. **Categorized words** (AI-isms vs Victorian vs other)

### Key Findings 🔍

**From the analysis above:**
- Top AI-signaling words revealed
- AI clichés detected (if present)
- Vocabulary-based vs context-based detection distinguished

### Next Steps 📋

**Part 2: The Findings**
- Vocabulary vs Structure: What's the primary signal?
- AI-isms analysis: Are they real?
- Victorian semantic gap: Quantify the temporal effect

**Part 3: Error Analysis**
- Find 3 human paragraphs misclassified as AI
- Analyze WHY the model failed
- Generate saliency maps for errors

---

**Next notebook:** `task3_findings_analysis.ipynb`

**Status:** Saliency Mapping ✅ COMPLETE